### Test case LVV-T1332 - Verify implementation of maximum time for retrieval of CCD-sized coadd cutouts
DMS-REQ-0377-V-02: Max time to retrieve single-CCD coadd cutout image<br>

To run as a scale test on data-int

### Specification: 
Specification: A CCD-sized cutout of a coadd, including mask and variance planes, shall be retrievable using the IVOA SODA protocol within ccdRetrievalTime with ccdRetrievalUsers simultaneous requests for distinct areas of the sky.

### Requirement Parameter
ccdRetrievalTime = 15[second] Maximum time allowed for retrieving a CCD-sized coadd cutout.,  <br>
ccdRetrievalUsers = 20[integer] Minimum number of simultaneous users retrieving a single CCD-sized coadd cutout.

In [ ]:
import os
import math
import numpy as np

from lsst.rsp.utils import get_pyvo_auth
from lsst.rsp import get_tap_service
import lsst.geom as geom
from lsst.afw.image import ExposureF 
from lsst.afw.fits import MemFileManager

from pyvo.dal.adhoc import DatalinkResults, SodaQuery
from astropy import units as u
from astropy.time import Time

import time

### Requirement Specs

In [ ]:
# Maximum time allowed for retrieving a CCD-sized coadd cutout., 
ccdRetrievalTime = 15  # seconds

# Minimum number of simultaneous users retrieving a single CCD-sized coadd cutout.
ccdRetrievalUsers = 50 

In [ ]:
# CCD-sized cutout  
pixels = 4000          #  4kx4k pixels / CCD
pixel_scale_arcsec = 0.2  # arcsec/pixel
pixel_scale_deg = pixel_scale_arcsec / 3600 

# CCD size in degrees
ccd_size_deg = pixels * pixel_scale_deg 
ccd_area_deg2 = ccd_size_deg ** 2
circle_radius_deg = math.sqrt(ccd_area_deg2 / math.pi)

print(f"CCD angular size: {ccd_size_deg:.2f}° × {ccd_size_deg:.2f}°")
print(f"CCD area: {ccd_area_deg2:.2f} deg²")
print(f"Equivalent circular radius: {circle_radius_deg:.2f}°")

### Setup services and functions

In [ ]:
service = get_tap_service("tap")
assert service is not None

### Generate an image cutout 
Use a DP1 r band image in the ECDFS field for a visit image that was obtained between MJD 60623.256 and 60623.259.

In [ ]:
# Sample region
ra_center = 53.1817060103852
dec_center = -27.770598997875638
lsst_band = 'r'

In [ ]:
# Circle cutout definition 
spherePoint = geom.SpherePoint(ra_center*geom.degrees, dec_center*geom.degrees)
radius = circle_radius_deg * u.deg

In [ ]:
# Query the ObsCore table to get the list of images satisfying these criteria
query = f"""
SELECT s_ra,s_dec, lsst_band, t_exptime,t_min,t_max,access_url
FROM ivoa.ObsCore 
WHERE CONTAINS(POINT('ICRS', {ra_center}, {dec_center}), s_region)=1
      AND obs_collection = 'LSST.DP1'
      AND dataproduct_type = 'image' AND instrument_name = 'LSSTComCam'
      AND dataproduct_subtype = 'lsst.deep_coadd'
      AND lsst_band = '{lsst_band}'
"""
print(query)

In [ ]:
results = service.search(query).to_table()

# Confirms the 
assert len(results) == 1

In [ ]:
results

In [ ]:
# Get the image URL and datalink result 
datalink_url = results[0].get('access_url')
dl_result = DatalinkResults.from_result_url(datalink_url,
                                            session=get_pyvo_auth())

f"Datalink status: {dl_result.status}. Datalink service url: {datalink_url}"

In [ ]:
# Set up the image cutout service
sq = SodaQuery.from_resource(dl_result,
                             dl_result.get_adhocservice_by_id("cutout-sync"),
                             session=get_pyvo_auth())

In [ ]:
# Define the circle stencil 
sq.circle = (spherePoint.getRa().asDegrees() * u.deg,
             spherePoint.getDec().asDegrees() * u.deg,
             radius)

### Record the time to retrieve one CCD-sized cutout
This is the code that needs to be scale tested for ccdRetrievalUsers (20) simultaneous users

In [ ]:
# This is the cell that needs to be benchmarked  for ccdRetrievalUsers (20) simultaneous users
start = time.time()
cutout_bytes = sq.execute_stream().read()
end = time.time()

In [ ]:
elapsed = end - start
print(f"Elapsed time: {elapsed:.4f} seconds")
assert elapsed < ccdRetrievalTime

In [ ]:
# Read into an ExposureF and check for the variance and mask planes -- why does this fail?
# mem = MemFileManager(len(cutout_bytes))
# mem.setData(cutout_bytes, len(cutout_bytes))
# exposure = ExposureF(mem)